Task 2: Develop Time Series Forecasting Models for All Datasets

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import warnings
warnings.filterwarnings("ignore")

tsla_data = pd.read_csv('../data/TSLA_data.csv', index_col='Date', parse_dates=True)
spy_data = pd.read_csv('../data/SPY_data.csv', index_col='Date', parse_dates=True)
bnd_data = pd.read_csv('../data/BND_data.csv', index_col='Date', parse_dates=True)


In [17]:
def preprocess_data(df):
    df = df[['Close']].copy()
    df.dropna(inplace=True)
    return df

tsla_data = preprocess_data(tsla_data)
spy_data = preprocess_data(spy_data)
bnd_data = preprocess_data(bnd_data)